In [8]:
!pip install transformers datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=fa968b993eef37633ecb4a83b1183a66ddaac427db1d23eaa64b1eaffa44cbc1
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10

In [1]:
# Activate GPU for faster training by clicking on 'Runtime' > 'Change runtime type' and then selecting GPU as the Hardware accelerator
# Then check if GPU is available
import torch
import pandas as pd
torch.cuda.is_available()

True

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
from datasets import load_dataset

wnut = load_dataset("wnut_17")

README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

wnut_17.py:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

The repository for wnut_17 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wnut_17.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [10]:
wnut["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [11]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

## Load Model

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

In [13]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 '@',
 'p',
 '##aul',
 '##walk',
 'It',
 "'",
 's',
 'the',
 'view',
 'from',
 'where',
 'I',
 "'",
 'm',
 'living',
 'for',
 'two',
 'weeks',
 '.',
 'Empire',
 'State',
 'Building',
 '=',
 'E',
 '##SB',
 '.',
 'Pretty',
 'bad',
 'storm',
 'here',
 'last',
 'evening',
 '.',
 '[SEP]']

In [14]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [15]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [16]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [17]:
import evaluate

seqeval = evaluate.load("seqeval")

In [18]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## traning

In [19]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [20]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "dslim/bert-base-NER", num_labels=13, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NER and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([13]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768])

In [26]:
training_args = TrainingArguments(
    output_dir="bert-base-NER-model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.325314,0.517564,0.409639,0.457320,0.947166
2,No log,0.309261,0.560102,0.405931,0.470715,0.948833


TrainOutput(global_step=426, training_loss=0.03266418260028105, metrics={'train_runtime': 128.2962, 'train_samples_per_second': 52.909, 'train_steps_per_second': 3.32, 'total_flos': 211741856300556.0, 'train_loss': 0.03266418260028105, 'epoch': 2.0})

In [27]:
trainer.push_to_hub()

events.out.tfevents.1731528908.2e6fc3f95e8a.1720.2:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/osmanh/bert-base-NER-model/commit/f6138b773cfde2d89c0f9922399f32e8b28c8631', commit_message='End of training', commit_description='', oid='f6138b773cfde2d89c0f9922399f32e8b28c8631', pr_url=None, repo_url=RepoUrl('https://huggingface.co/osmanh/bert-base-NER-model', endpoint='https://huggingface.co', repo_type='model', repo_id='osmanh/bert-base-NER-model'), pr_revision=None, pr_num=None)

In [28]:
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

In [29]:
from transformers import pipeline

classifier = pipeline("ner", model="osmanh/bert-base-NER-model")
classifier(text)

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity': 'B-group',
  'score': 0.76773465,
  'index': 1,
  'word': 'The',
  'start': 0,
  'end': 3},
 {'entity': 'B-group',
  'score': 0.88921046,
  'index': 2,
  'word': 'Golden',
  'start': 4,
  'end': 10},
 {'entity': 'I-group',
  'score': 0.96675247,
  'index': 3,
  'word': 'State',
  'start': 11,
  'end': 16},
 {'entity': 'I-group',
  'score': 0.9660521,
  'index': 4,
  'word': 'Warriors',
  'start': 17,
  'end': 25},
 {'entity': 'B-location',
  'score': 0.9861946,
  'index': 13,
  'word': 'San',
  'start': 80,
  'end': 83},
 {'entity': 'I-location',
  'score': 0.9853389,
  'index': 14,
  'word': 'Francisco',
  'start': 84,
  'end': 93}]

##  manually replicate the results

In [30]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("osmanh/bert-base-NER-model")
inputs = tokenizer(text, return_tensors="pt")

In [32]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("osmanh/bert-base-NER-model")
with torch.no_grad():
    logits = model(**inputs).logits

In [33]:
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
predicted_token_class

['O',
 'B-group',
 'B-group',
 'I-group',
 'I-group',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'I-location',
 'O',
 'O']

In [34]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("osmanh/bert-base-NER-model")
inputs = tokenizer(text, return_tensors="tf")

In [36]:
inputs

{'input_ids': <tf.Tensor: shape=(1, 17), dtype=int32, numpy=
array([[ 101, 1109, 3684, 1426, 9526, 1132, 1126, 1237, 1848, 3163, 1264,
        1359, 1107, 1727, 2948,  119,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 17), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 17), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [35]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained("osmanh/bert-base-NER-model")
logits = model(**inputs).logits

All PyTorch model weights were used when initializing TFBertForTokenClassification.

All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [37]:
logits

<tf.Tensor: shape=(1, 17, 13), dtype=float32, numpy=
array([[[ 5.7522941e+00, -1.5269487e+00, -1.3250411e+00, -7.3084849e-01,
         -4.2769969e-01, -4.2823792e-01, -9.7752130e-01,  6.4284362e-02,
         -1.1987450e+00, -4.1784003e-02, -1.3114092e+00, -9.9459738e-01,
         -1.1891267e+00],
        [ 1.1760787e+00, -9.4354999e-01, -8.3611429e-01, -3.5450798e-01,
         -1.9836677e+00,  3.7022932e+00,  2.1100636e-01,  1.4451417e+00,
         -1.0525566e+00, -4.1120476e-01, -8.3896923e-01, -1.3745571e+00,
         -1.6763508e+00],
        [-1.7217441e+00, -1.4767167e-01,  1.9165315e-01, -4.7454226e-01,
         -1.5391603e+00,  5.3112898e+00,  2.9226542e+00,  1.6838843e-01,
         -2.5951326e-01, -7.5277054e-01, -7.7576423e-01, -1.0051383e+00,
         -1.1654317e+00],
        [-1.1263127e+00, -1.5078442e+00,  1.8774619e+00, -1.1726873e+00,
         -9.7431794e-02,  2.9706681e-01,  6.0267196e+00, -9.0458483e-01,
          5.8589786e-01, -7.2848123e-01, -2.6855311e-01, -1.251571

In [39]:
import tensorflow as tf

predicted_token_class_ids = tf.math.argmax(logits, axis=-1)
predicted_token_class = [model.config.id2label[t] for t in predicted_token_class_ids[0].numpy().tolist()]
predicted_token_class

['O',
 'B-group',
 'B-group',
 'I-group',
 'I-group',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'I-location',
 'O',
 'O']